In [11]:
from utils import *

In [12]:
#snapshots = binarytoData('snapshotsDict.pickle')

In [16]:
nodeLabels = []
def createDistanceMatrix():
    df = pd.read_csv('Dados/snapshots_independent.csv', usecols=['REF','P1','P2','P3','P4','P5','P6',
                                                                 'P7','P8','P9','P10','P11','P12'])
    df.dropna(inplace = True)
    
    nodeLabels = list(df['REF'])
    
    alsfrsList = []
    refList = []
    for i in df.index:
        l = list(df.loc[i])
        if l[0] not in refList:
            refList.append(l[0])
            l.pop(0)
            alsfrsList.append(l)
            
    listSize = len(alsfrsList)
    matrixSize = listSize*listSize
    print(matrixSize)
    distanceMatrix = np.array([0.0]*matrixSize).reshape(listSize,listSize)
    
    c = 0
    for i in range(listSize):
        for j in range(listSize):
            similarity = cosine_similarity([alsfrsList[i]], [alsfrsList[j]])[0][0]
            distanceMatrix[i][j] = similarity
            c = c + 1
            if (c % 100000) == 0:
                print(c)
        
    return distanceMatrix
    
#st = time.time()
#distanceMatrix = createDistanceMatrix()
#et = time.time()
#elapsed_time = et - st
#print('Execution time:', elapsed_time, 'seconds')

In [15]:
#print(distanceMatrix)
#dataToBinary(distanceMatrix, 'distanceMatrix_PT_S1.pickle')

[[1.         0.76041282 0.947982   ... 0.98384    0.83319558 0.95692307]
 [0.76041282 1.         0.82793953 ... 0.78004736 0.6008016  0.80817931]
 [0.947982   0.82793953 1.         ... 0.93119969 0.90642326 0.96186517]
 ...
 [0.98384    0.78004736 0.93119969 ... 1.         0.7823005  0.96708402]
 [0.83319558 0.6008016  0.90642326 ... 0.7823005  1.         0.81206897]
 [0.95692307 0.80817931 0.96186517 ... 0.96708402 0.81206897 1.        ]]
